# Package 설치

In [1]:
%pip install transformers tokenizers datasets accelerate  -qU

Note: you may need to restart the kernel to use updated packages.


# Hugging Face Pipeline을 이용한 모델 활용

- Pipeline은 Transformers 라이브러리의 가장 기본적인 객체로, **전처리 - 추론 -> 후처리** 로 이어지는 일련의 과정을 자동화하여 손쉽게 모델을 사용할 수 있게 해준다.
- Task에 따라 다양한 Pipeline 클래스를 제공하며 `pipeline` 함수를 이용해 쉽게 생성할 수 있다.
- **task만 지정**해서 기본 제공 모델과 토크나이저를 사용하거나 **직접 모델과 토크나이저를 지정**해 생성할 수 있다.
- https://huggingface.co/docs/transformers/pipeline_tutorial

![huggingface_pipeline.png](figures/huggingface_pipeline.png)

## 지원하는 주요 태스크
- https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task
### 자연어 처리 태스크
- **text-classification**: 텍스트 분류
- **text-generation**: 텍스트 생성
- **translation**: 번역
- **summarization**: 요약
- **question-answering**: 질의응답
- **fill-mask**: 마스크 토큰 채우기
- **token-classification**: 개체명 인식, Pos tagging 같이 개별 토큰에 대한 분류
- **feature-extraction**: 특징 추출(context vector)

### 영상 처리 태스크
- **image-classification**: 이미지 분류
- **object-detection**
  -  객체 검출 (Object Detection)
  -  이미지 안에서 객체들의 위치와 class를 찾아내는 작업
- **image-segmentation**
  -  이미지 세분화 (Image Segmentation)
  -  이미지를 픽셀 단위로 분할하여 각 픽셀이 어떤 객체에 속하는지 분류하는 작업

## 모델 검색
![huggingface_model_search.png](figures/huggingface_model_search.png)



## pipeline 함수
- 주요파라미터
  - **task:** 수행하려는 작업의 유형을 문자열로 지정한다.
  - **model:**
    - 사용할 사전 학습된 모델의 이름 또는 경로를 지정한다. 
    - 모델이름(ID)은 `[모델소유자이름]/[모델이름]` 형식이다. Hugging Face에서 제공하는 모델의 경우는 `모델소유자이름`이 생략되어 있다. (ex: "google/gemma-2-2b", "gpt2")
    - 모델을 명시적으로 지정하지 않으면, **task에 맞는 기본 모델이 로드**된다.
  - **tokenizer:** 자연어 task에서 사용할 토크나이저를 지정한다. 생략하면 모델과 같이 제공되는(model과 이름이 같은 토크나이저) 토크나이저를 사용한다.
  - **framework:** 사용할 딥러닝 프레임워크를 지정한다. 'pt'는 PyTorch(Default), 'tf'는 TensorFlow를 지정한다.
  - **device:** Pipeline 모델을 실행할 디바이스를 지정한다. 문자열로 `"cpu", "cuda:1", "mps"`, 또는 GPU 번호를 정수로 지정한다. 
  - **revision:** 모델의 특정 버전을 지정할 때 사용한다.
  - **trust_remote_code:** hub 모델을 직접 다운 받는 것이 아니라 모델을 다운 받는 **코드**를 다운 받아 local에서 실행하는 경우 코드를 실행할 수있게 할 지 여부. (bool)
  - **use_fast:** 
    - 빠른 토크나이저를 사용할지 여부를 지정합니다. 기본값은 True입니다.
    - 빠른 토크나이저는 `Rust` 언어로 구현되어 속도가 빠르다. 단 모든 모델에 대해 지원하지 않는다. 지원하지 않을 경우 `use_fast=True`로 설정해도 일반 토크나이저가 사용된다.

## Task 별 pipeline 실습

### 텍스트 분류

In [ ]:
data = [ 
    "The project was completed successfully.", 
    "She always brings positive energy to the team.", 
    "I am confident that we will achieve our goals.",
    "The results were not as expected.", 
    "He struggled to meet the deadline.", 
    "The client was dissatisfied with the final product." 
]

In [ ]:
model="distilbert-base-uncased-finetuned-sst-2-english"

In [ ]:
kor_texts = [
    "이 영화 정말 재미있어요!",
    "서비스가 별로였어요.",
    "제품 품질이 우수합니다.",
    "따듯하고 부드럽고 제품은 너무 좋습니다. 그런데 배송이 너무 늦네요."  # 애매한 것 0.56 정도 나오네.
]

In [ ]:
model = 'Copycats/koelectra-base-v3-generalized-sentiment-analysis' 

### 제로샷 분류
- 제로샷(Zero-shot)은 각 개별 작업에 대한 특정 교육 없이 작업을 수행할 수 있는 task다.
- 입력 텍스트와 함께 클래스 레이블을 제공하면 분류 작업을 한다.
- 모델은  `task`에서 `Zero-Shot` 으로 시작하는 task를 선택하여 검색한다.

In [ ]:
model = "facebook/bart-large-mnli"

text = ["Python is a programming language.", 
        "I love soccer", 
        "The stock price rose slightly today."]

### 텍스트 생성

In [ ]:
start_text = "Today weather"

### 마스크 채우기

In [ ]:
text = "I'm going to <mask> because <mask> am hurt."
model="distilroberta-base"

In [ ]:
text = "오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK]가 내리겠고, 기온이 내려가면서 점차 [MASK]이 오는 곳이 많겠습니다"
model='beomi/kcbert-base'

### Token별 분류
- task: token-classification 
  - 개체명인식(ner), 품사부착(pos tagging)을 수행하는 task 
  - 개체명 인식은 문장에서 특정한 개체명(예: 사람 이름, 지명, 조직명 등)을 식별하는 task이다. 

In [ ]:
text = "My name is Sylvain and I work at Hugging Face in Brooklyn."
model = "dbmdz/bert-large-cased-finetuned-conll03-english"

### 질의 응답
- 문서와 질문을 주면 문서에서 답을 찾아 응답한다.

In [ ]:
model = "distilbert-base-cased-distilled-squad"

question="Where do I work?"
# question="Where is Hugging Face?"
context="My name is Sylvain and I work at Hugging Face in Brooklyn"

In [ ]:
context = """Sri Lankans are voting for a new president in the first election since mass protests sparked by the country's worst-ever economic crisis unseated the leader in 2022.
Saturday's vote is widely regarded as a referendum on economic reforms meant to put the country on the road to recovery.
But many are still struggling to make ends meet because of tax hikes, and cuts to subsidies and welfare.
Multiple analysts predict that economic concerns will be front of mind for voters in what is shaping up to be a close race.
"The country's soaring inflation, skyrocketing cost-of-living and poverty have left the electorate desperate for solutions to stabilise prices and improve livelihoods," Soumya Bhowmick, an associate fellow at India-based think tank the Observer Research Foundation, told the BBC.
"With the country seeking to emerge from its economic collapse, this election serves as a crucial moment for shaping Sri Lanka’s recovery trajectory and restoring both domestic and international confidence in its governance."
President Ranil Wickremesinghe, who was charged with the monumental task of leading Sri Lanka out of its economic collapse, is seeking another term.
The 75-year-old was appointed by parliament a week after former president Gotabaya Rajapaksa was chased out of power.
Shortly after taking office, Wickremesinghe crushed what was left of the protest movement. He has also been accused of shielding the Rajapaksa family from prosecution and allowing them to regroup - allegations he has denied.
Another strong contender is leftist politician Anura Kumara Dissanayake, whose anti-corruption platform has seen him draw increasing public support.
More candidates are running in Saturday's election than any other in Sri Lanka's history. But of more than three dozen, four are dominating the limelight.
Other than Wickremesinghe and Dissanayake, there is also the leader of the opposition, Sajith Premadasa, and the 38-year-old nephew of the ousted president, Namal Rajapaksa.
Counting begins once polls close at 16:00 local time (10:30 GMT), but results are not expected to become clear until Sunday morning."""

question="How many candidates are there in total?"
# question="When will the results be known?" 

### 문서 요약

In [ ]:
model = "summarization","sshleifer/distilbart-cnn-12-6"

### 번역

In [ ]:
model = "Helsinki-NLP/opus-mt-fr-en"
text = "Ce cours est produit par Hugging Face."

In [ ]:
model = "Helsinki-NLP/opus-mt-ko-en"

### 이미지를 설명하는 텍스트 생성

In [ ]:
url = "https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png"
url = "https://th.bing.com/th?id=ORMS.c526884bbea37c0bb9501f4f83b601e4&pid=Wdp&w=268&h=140&qlt=90&c=1&rs=1&dpr=1&p=0"
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [ ]:
model = "ydshieh/vit-gpt2-coco-en"

### 이미지 분류

In [ ]:
url = "https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/202306/25/488f9638-800c-4bac-ad65-82877fbff79b.jpg"

In [ ]:
model = "google/vit-base-patch16-224"

### Object Detection

In [ ]:
image_path = r"data/image1.jpg"
image_path = r"data/image2.jpg"
image_path = r"data/image3.jpg"

model='facebook/detr-resnet-50'